<a href="https://colab.research.google.com/github/RobolinkInc/zumi/blob/master/sample/deep-learning-demos/tourist/Z%C3%BCmi_3_Landmarks_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#How to Control a Robot with a NN

This robot has an image sensor, and 4 arrow keys for control. We will control the robot manually first, then log the image, and inputs at each moment. We will use these as examples for our NN to reproduce.

This will make a model capable of running and training on a PiZero.

## First define a NN

In [0]:
import keras

keras.backend.clear_session()

In [0]:
image_shape = (64,64,3) #color image
# image_shape = (32,32,1) #monochrome image
num_outputs = 6  # left, right, up, eiffel, chicago, start


import keras
from keras.layers import Input, Conv2D, Dense, Flatten, Lambda
from keras.layers.normalization import BatchNormalization

def make_model():
  '''
  define the layer definitions, inputs, and outputs of the NN
  '''
  img_in = Input(shape=image_shape)
  x = img_in
#   x = Lambda(lambda x : x / 255.0 )(x)
  x = BatchNormalization()(x)
  x = Flatten()(x)
  x = Dense(64, activation='relu')(x)
  x = Dense(32, activation='relu')(x)
  x = Dense(num_outputs, activation='softmax')(x)

  model = keras.models.Model(inputs=[img_in], outputs=[x])
  model.compile(optimizer="adam", metrics=['acc'], loss='categorical_crossentropy')
  return model

model = make_model()
print(model.summary())

##Clear out the previous set of data from Google Drive

`python3 /home/pi/zumi/sample/deep-learning-demos/driving/ClearImagesFromCloud.py --noauth_local_webserver`

##Record some data.

Run this script on Zumi: 

`python3 /home/pi/zumi/sample/deep-learning-demos/tourist/train_landmark.py`

Use W, A, and S to control the robot and Z when you want to stop at the landmark.

Upload the training data to [this folder](https://drive.google.com/drive/u/3/folders/1NwZ7PWLM4ZlHcvZhEWAM2N7YJDPhM9Pt) on Google Drive

##Upload data from Zumi to Google Drive

`python3 /home/pi/zumi/sample/deep-learning-demos/driving/SaveImagesToCloud.py --noauth_local_webserver`



#Load images from Google Drive

In [0]:
from google.colab import drive
drive.mount("/content/drive")

!ls "/content/drive/My Drive/Zumi-Training-Data"

In [0]:
import cv2
import numpy 

def normalize(image):
  maxValue = image.max()
  scaledValue = 255.0/image.max()
  normalizedImage = image*scaledValue
  return normalizedImage

import os
files = os.listdir('/content/drive/My Drive/Zumi-Training-Data')
print("Loading " + str(len(files)) + " files...")

images = []
commands = []
up_images = 0
right_images = 0
left_images = 0
eiffel_images = 0
chicago_images = 0
start_images = 0

for file_name in files:
  image = cv2.imread('/content/drive/My Drive/Zumi-Training-Data/' + file_name)
  images.append(image)
  
  filename_pieces = file_name.split(".")
  command = filename_pieces[2]

  command_array = numpy.zeros(num_outputs)
  if(command == "up"): 
    up_images += 1
    arrow_index = 0
  elif(command == "left"): 
    arrow_index = 1
    left_images += 1
  elif(command == "right"): 
    arrow_index = 2
    right_images += 1
  elif(command == "eiffel"): 
    arrow_index = 3
    eiffel_images += 1
  elif(command == "chicago"): 
    arrow_index = 4
    chicago_images += 1
  elif(command == "start"): 
    arrow_index = 5
    start_images += 1
  else:
    print("Invalid command: " + command)
    
  command_array[arrow_index] = 1.0
  commands.append(command_array)
  
print("Uploaded " + str(len(images)) + " images, and " + str(len(commands)) + " commands.")
print(str(up_images) + " were UP. " + str(left_images) + " were LEFT. " + str(right_images) + " were RIGHT. " + str(eiffel_images) + " were EIFFEL. " + str(chicago_images) + " were CHICAGO. " + str(start_images) + " were START.")

#duplicate data so we can use fewer epochs
for i in range(2):
  images = images + images
  commands = commands + commands

images = numpy.array(images)
commands = numpy.array(commands)



In [0]:
images.shape

(10712, 64, 64, 3)

## Train the NN to reproduce the controls given the inputs

you can increase epoch count until your accuracy stops improving

In [0]:
model.fit(images, commands, epochs=10)

## Save and download the model

In [0]:
import json
json_string = model.to_json()
with open('landmark_model.json', 'w') as outfile:  
    json.dump(json_string, outfile)
from google.colab import files
files.download('landmark_model.json')

model.save_weights('landmark_weights.h5')
from google.colab import files
files.download('landmark_weights.h5')

In [0]:
import numpy as np

def eval(iSample):
  x = images[iSample] #cv2.imread('/content/drive/My Drive/Zumi-Training-Data/78.left.jpg')
  y_prob = model.predict(x[None, :, :, :])
  print("Sample", iSample)
  print(y_prob)
  print(commands[iSample])
  y_classes = np.argmax(y_prob[0])
  print(y_classes)
  
for i in range(5):
  eval(i)
  


## Send to Pi0

scp -r /home/joonspoon/Downloads/landmark_weights.h5 pi@june-zumi.local:/home/pi/zumi/sample/deep-learning-demos/tourist/

scp -r /home/joonspoon/Downloads/landmark_model.json pi@june-zumi.local:/home/pi/zumi/sample/deep-learning-demos/tourist/

## Use to control robot

By running [this notebook](https://github.com/RobolinkInc/zumi/tree/master/sample/deep-learning-demos/driving/ControlUsingModel.ipynb)  on the robot.

http://june-zumi.local:5555/notebooks/sample/deep-learning-demos/driving/ControlUsingModel.ipynb

